# **Read mapping using post-QC NGS data**
---

### 1. Reference genome

University of Californa Santa Cruz is a common source for reference genomes ranging  mulitple species including human. To browse their well-annotated directory, visit: https://hgdownload.soe.ucsc.edu/goldenPath/hg38/bigZips/

For our purposes, we are downloading hg38 assesmbly which is the latest GRCh assembly and more commonly used assembly than the latest hs1 genome hosted on UC Santa Cruz.

In [2]:
# download the whole genome reference genome
!wget https://hgdownload.soe.ucsc.edu/goldenPath/hg38/bigZips/latest/hg38.fa.gz

--2024-10-16 19:34:13--  https://hgdownload.soe.ucsc.edu/goldenPath/hg38/bigZips/latest/hg38.fa.gz
Resolving hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)... 128.114.119.163
Connecting to hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)|128.114.119.163|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1012013082 (965M) [application/x-gzip]
Saving to: ‘hg38.fa.gz’

hg38.fa.gz          100%[===================>] 965.13M  58.5MB/s    in 17s     

2024-10-16 19:34:31 (57.6 MB/s) - ‘hg38.fa.gz’ saved [1012013082/1012013082]



In [3]:
# move file
!mkdir ./files/step_2_read_mapping
!mv hg38.fa.gz ./files/step_2_read_mapping

In [4]:
# view the reference genome
!zcat ./files/step_2_read_mapping/hg38.fa.gz| head -n10

>chr1
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN

gzip: stdout: Broken pipe


A quick glance at the assembly shows that it does not only include the canonical nucleotide ATGC, but also other symbols. The following explanation is given on UCSC website:
> hg38.fa.gz - "Soft-masked" assembly sequence in one file.
    Repeats from RepeatMasker and Tandem Repeats Finder (with period of 12 or
    less) are shown in lower case; non-repeating sequence is shown in upper
    case. (again, the most current version of this file is latest/hg38.fa.gz)

Thus, we can infer that the N is probably gaps of unknown nucleotides and not hard-masked repeats such as the case in another file:
> hg38.fa.masked.gz - "Hard-masked" assembly sequence in one file.
    Repeats are masked by capital Ns; non-repeating sequence is shown in
    upper case.

Let's look at another section of the sequence:

In [8]:
!zcat ./files/step_2_read_mapping/hg38.fa.gz| head -n30000 | tail

ggtcttaaactcctgacctcatgatccgcactcggcctcccaaagtgctg
ggattacaggcgtgagccaccgtgcccagccATAGATAGATATTATGTCT
TCTATTCACCCTGGTGGGGTTGGTTCCAAACAGCTCtttttttttttttt
tttttttttttgagatggagttttgccttgtagcccaggctggagtgcag
tggcgcaatctctggtcactgcaacctccgcctgccgggttcaagcgatt
ctccttcctcagccttcccaagtagctgggattataggcatgcgccacca
agcccggctaattttgtatttttagtagagatggggtttcaccaggttgg
tcaggctggtctcgaactcccgacgtcaggtgatccacccacctcggcct
cccgaagtgccgggattacaggcgtgagccaccgtgcccagccttttttt
ttttttctttttgagacagagtcttgctctgttgcccaggctggagtcgt

gzip: stdout: Broken pipe


Here, we can see the upper-case and lower-case nucleotides representing non-repeating sequence and soft-masked repeating sequence repsectively as noted in the explanation above.

---

### 2. Tools for read mapping

There are multiple tools available for read mapping, and a comparison of some common tools can be found in this study https://doi.org/10.3390/plants9040439

In this document, we will use two (2) tools:
- bwa-mem https://github.com/lh3/bwa
- minimap2 https://github.com/lh3/minimap2

Note that minimap2 is created by the same people who made bwa-mem but intended for PacBio and NanoPore (long) read mapping. Also, bew-mem2 is in beta for an upgraded version of the current bwa-mem.

bwa-mem can be installed with conda install:

In [3]:
!conda install -c bioconda bwa -y

Retrieving notices: ...working... done
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.3.1
  latest version: 24.9.2

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=24.9.2



## Package Plan ##

  environment location: /home/tnguyen623/anaconda3/envs/rep_binf

  added / updated specs:
    - bwa


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    bwa-0.7.17                 |       h5bf99c6_8         608 KB  bioconda
    ------------------------------------------------------------
                                           Total:         608 KB

The following NEW packages will be INSTALLED:

  bwa                bioconda/linux-64::bwa-0.7.17-h5bf99c6_8 



                                                             

In [4]:
# checking the download finished correctly and that we have access to bwa-mem
!which bwa

~/anaconda3/envs/rep_binf/bin/bwa


In [6]:
!bwa


Program: bwa (alignment via Burrows-Wheeler transformation)
Version: 0.7.17-r1188
Contact: Heng Li <lh3@sanger.ac.uk>

Usage:   bwa <command> [options]

Command: index         index sequences in the FASTA format
         mem           BWA-MEM algorithm
         fastmap       identify super-maximal exact matches
         pemerge       merge overlapping paired ends (EXPERIMENTAL)
         aln           gapped/ungapped alignment
         samse         generate alignment (single ended)
         sampe         generate alignment (paired ended)
         bwasw         BWA-SW for long queries

         shm           manage indices in shared memory
         fa2pac        convert FASTA to PAC format
         pac2bwt       generate BWT from PAC
         pac2bwtgen    alternative algorithm for generating BWT
         bwtupdate     update .bwt to the new format
         bwt2sa        generate SA from BWT and Occ

Note: To use BWA, you need to first index the genome with `bwa index'.
      There are

minimap2 can be installed in the same way:

In [7]:
!conda install -c bioconda minimap2 -y

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.3.1
  latest version: 24.9.2

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=24.9.2



## Package Plan ##

  environment location: /home/tnguyen623/anaconda3/envs/rep_binf

  added / updated specs:
    - minimap2


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    minimap2-2.17              |       h8b12597_1         2.0 MB  bioconda
    ------------------------------------------------------------
                                           Total:         2.0 MB

The following NEW packages will be INSTALLED:

  minimap2           bioconda/linux-64::minimap2-2.17-h8b12597_1 



                                                                                
Preparing t

In [9]:
!which minimap2

~/anaconda3/envs/rep_binf/bin/minimap2


In [10]:
!minimap2

Usage: minimap2 [options] <target.fa>|<target.idx> [query.fa] [...]
Options:
  Indexing:
    -H           use homopolymer-compressed k-mer (preferrable for PacBio)
    -k INT       k-mer size (no larger than 28) [15]
    -w INT       minimizer window size [10]
    -I NUM       split index for every ~NUM input bases [4G]
    -d FILE      dump index to FILE []
  Mapping:
    -f FLOAT     filter out top FLOAT fraction of repetitive minimizers [0.0002]
    -g NUM       stop chain enlongation if there are no minimizers in INT-bp [5000]
    -G NUM       max intron length (effective with -xsplice; changing -r) [200k]
    -F NUM       max fragment length (effective with -xsr or in the fragment mode) [800]
    -r NUM       bandwidth used in chaining and DP-based alignment [500]
    -n INT       minimal number of minimizers on a chain [3]
    -m INT       minimal chaining score (matching bases minus log gap penalty) [40]
    -X           skip self and dual mappings (for the all-vs-all mode)
    

---

### 3. Read mapping with bwa-mem

The bwa-mem quick-start guide tells us that it requires the indexing of the reference genome. This makes alligning faster and save memory when doing so by creating a searchable structure of the original reference genome.

In [13]:
# index the reference genome
!bwa index


Usage:   bwa index [options] <in.fasta>

Options: -a STR    BWT construction algorithm: bwtsw, is or rb2 [auto]
         -p STR    prefix of the index [same as fasta name]
         -b INT    block size for the bwtsw algorithm (effective with -a bwtsw) [10000000]
         -6        index files named as <in.fasta>.64.* instead of <in.fasta>.* 

         `-a div' do not work not for long genomes.



In [15]:
!bwa index -a rb2 ./files/step_2_read_mapping/hg38.fa.gz

[bwa_index] Pack FASTA... 31.59 sec
[bwa_index] Construct BWT for the packed sequence...
^C


**At this point**, the whole genome reference genome is too big for a demonstration. Therefore, we will pick a small chromosome to use as a reference genome for this demonstration.

In [16]:
# download the reference chromosome 19
!wget https://hgdownload.soe.ucsc.edu/goldenPath/hg38/chromosomes/chr19.fa.gz
# move the file
!mv chr19.fa.gz ./files/step_2_read_mapping

--2024-10-18 14:58:14--  https://hgdownload.soe.ucsc.edu/goldenPath/hg38/chromosomes/chr19.fa.gz
Resolving hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)... 128.114.119.163
Connecting to hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)|128.114.119.163|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17916986 (17M) [application/x-gzip]
Saving to: ‘chr19.fa.gz’

chr19.fa.gz         100%[===================>]  17.09M  25.0MB/s    in 0.7s    

2024-10-18 14:58:15 (25.0 MB/s) - ‘chr19.fa.gz’ saved [17916986/17916986]



In [17]:
# index the chromosome 19 reference
!bwa index -a rb2 ./files/step_2_read_mapping/chr19.fa.gz

[bwa_index] Pack FASTA... 0.58 sec
[bwa_index] Construct BWT for the packed sequence...
[bwa_index] 47.12 seconds elapse.
[bwa_index] Update BWT... 0.26 sec
[bwa_index] Pack forward-only FASTA... 0.41 sec
[bwa_index] Construct SA from BWT and Occ... 11.60 sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa index -a rb2 ./files/step_2_read_mapping/chr19.fa.gz
[main] Real time: 60.858 sec; CPU: 59.984 sec


In [19]:
# mapping with bwa
!bwa mem


Usage: bwa mem [options] <idxbase> <in1.fq> [in2.fq]

Algorithm options:

       -t INT        number of threads [1]
       -k INT        minimum seed length [19]
       -w INT        band width for banded alignment [100]
       -d INT        off-diagonal X-dropoff [100]
       -r FLOAT      look for internal seeds inside a seed longer than {-k} * FLOAT [1.5]
       -y INT        seed occurrence for the 3rd round seeding [20]
       -c INT        skip seeds with more than INT occurrences [500]
       -D FLOAT      drop chains shorter than FLOAT fraction of the longest overlapping chain [0.50]
       -W INT        discard a chain if seeded bases shorter than INT [0]
       -m INT        perform at most INT rounds of mate rescues for each read [50]
       -S            skip mate rescue
       -P            skip pairing; mate rescue performed unless -S also in use

Scoring options:

       -A INT        score for a sequence match, which scales options -TdBOELU unless overridden [1]
     

In [20]:
!bwa mem -t 24 ./files/step_2_read_mapping/chr19.fa.gz ./files/step_1_ngs_quality_control/QC_by_fastp/QC_SRR075914_1.fastq.gz ./files/step_1_ngs_quality_control/QC_by_fastp/QC_SRR075914_2.fastq.gz > ./files/step_2_read_mapping/SRR075914_chr19.sam

[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 1291600 sequences (87742908 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (1, 20329, 12, 1)
[M::mem_pestat] skip orientation FF as there are not enough pairs
[M::mem_pestat] analyzing insert size distribution for orientation FR...
[M::mem_pestat] (25, 50, 75) percentile: (107, 141, 186)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (1, 344)
[M::mem_pestat] mean and std.dev: (148.24, 57.46)
[M::mem_pestat] low and high boundaries for proper pairs: (1, 423)
[M::mem_pestat] analyzing insert size distribution for orientation RF...
[M::mem_pestat] (25, 50, 75) percentile: (136, 3552, 7067)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (1, 20929)
[M::mem_pestat] mean and std.dev: (3298.75, 3286.77)
[M::mem_pestat] low and high boundaries for proper pairs: (1, 27860)
[M::mem_pestat] skip orientation RR as there are not enough pairs
[M::mem_pestat] skip o

Checking the output sequence alignment/map format (SAM) file:

In [23]:
!head ./files/step_2_read_mapping/SRR075914_chr19.sam

@SQ	SN:chr19	LN:58617616
@PG	ID:bwa	PN:bwa	VN:0.7.17-r1188	CL:bwa mem -t 24 ./files/step_2_read_mapping/chr19.fa.gz ./files/step_1_ngs_quality_control/QC_by_fastp/QC_SRR075914_1.fastq.gz ./files/step_1_ngs_quality_control/QC_by_fastp/QC_SRR075914_2.fastq.gz
SRR075914.4	77	*	0	0	*	*	0	0	TTTNAATGTATAGTTCAATGAATTTTGACAAAGAAACACACTCATGTAACCACTACAATCAAGACATN	7;;!7?6=3<C=C@C-B==<DBDDCC???C=@=CA6;@BD;>C########################!	AS:i:0	XS:i:0
SRR075914.4	141	*	0	0	*	*	0	0	TGGGTGGGAACACAGCCAAACCATATTAGGGGCACAAGAGAACTTTCAGGTATGATAGANATGTTNNN	A>>:;CA=:C5DD5DEE??EEBAEED5BDA)*567=DDCA5A<<?AAA5:==C######!#####!!!	AS:i:0	XS:i:0
SRR075914.5	77	*	0	0	*	*	0	0	TTGNAGCTGGAATCTGGTCCTGCCTCCTGTCCACCATGATCATTTGCCAAGTTTTTAATGGGGTTGTN	@AA!=CCCCAFFGFGGGDBEEC?BEDEBDDEFDEDBEDBDFA?EFDD?E?C=:AB############!	AS:i:0	XS:i:0
SRR075914.5	141	*	0	0	*	*	0	0	TATAACAATCATGCAAGAGAGCTTCTATTGTTATCATCCCCACTTCACAGAGAATAAAACTGAGACAN	DGGFDGGGGDGGGAGD?A?:A<AA?FFFDDCD?CA>?-A(AA:A?EB-=BCBBD-CB##########!	AS:i:0	XS:i:0
SRR075914.13	77	

In [22]:
!wc -l ./files/step_2_read_mapping/SRR075914_chr19.sam

1292514 ./files/step_2_read_mapping/SRR075914_chr19.sam


The output of the read mapping process is a sequence alignment/map format (SAM) file. This type of file compactly stores a sequence of multiple reads aligned to a reference sequence.

---

### 3. Read mapping with minimap2

Indexing of the reference genome is also required for minimap2, and we will do this using minimap2 for the chr19 reference chromosome:

In [1]:
!minimap2

Usage: minimap2 [options] <target.fa>|<target.idx> [query.fa] [...]
Options:
  Indexing:
    -H           use homopolymer-compressed k-mer (preferrable for PacBio)
    -k INT       k-mer size (no larger than 28) [15]
    -w INT       minimizer window size [10]
    -I NUM       split index for every ~NUM input bases [4G]
    -d FILE      dump index to FILE []
  Mapping:
    -f FLOAT     filter out top FLOAT fraction of repetitive minimizers [0.0002]
    -g NUM       stop chain enlongation if there are no minimizers in INT-bp [5000]
    -G NUM       max intron length (effective with -xsplice; changing -r) [200k]
    -F NUM       max fragment length (effective with -xsr or in the fragment mode) [800]
    -r NUM       bandwidth used in chaining and DP-based alignment [500]
    -n INT       minimal number of minimizers on a chain [3]
    -m INT       minimal chaining score (matching bases minus log gap penalty) [40]
    -X           skip self and dual mappings (for the all-vs-all mode)
    

In [2]:
# index reference chromosome 19 with minimap2
!minimap2 -d ./files/step_2_read_mapping/chr19_minimap2_index ./files/step_2_read_mapping/chr19.fa.gz

[M::mm_idx_gen::1.766*1.00] collected minimizers
[M::mm_idx_gen::2.071*1.29] sorted minimizers
[M::main::2.335*1.26] loaded/built the index for 1 target sequence(s)
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 1
[M::mm_idx_stat::2.422*1.25] distinct minimizers: 6639965 (86.42% are singletons); average occurrences: 1.666; average spacing: 5.298
[M::main] Version: 2.17-r941
[M::main] CMD: minimap2 -d ./files/step_2_read_mapping/chr19_minimap2_index ./files/step_2_read_mapping/chr19.fa.gz
[M::main] Real time: 2.434 sec; CPU: 3.036 sec; Peak RSS: 0.514 GB


In [3]:
# read mapping with minimap2 with options "-c -a" so that it outputs standard SAM file
!minimap2 -c -a ./files/step_2_read_mapping/chr19_minimap2_index ./files/step_1_ngs_quality_control/QC_by_fastp/./files/step_1_ngs_quality_control/QC_by_fastp/QC_SRR075914_1.fastq.gz ./files/step_1_ngs_quality_control/QC_by_fastp/QC_SRR075914_2.fastq.gz > ./files/step_2_read_mapping/SRR075914_chr19_minimap2.sam

[M::main::0.481*1.00] loaded/built the index for 1 target sequence(s)
[M::mm_mapopt_update::0.613*1.00] mid_occ = 279
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 1
[M::mm_idx_stat::0.709*1.00] distinct minimizers: 6639965 (86.42% are singletons); average occurrences: 1.666; average spacing: 5.298
ERROR: failed to open file './files/step_1_ngs_quality_control/QC_by_fastp/./files/step_1_ngs_quality_control/QC_by_fastp/QC_SRR075914_1.fastq.gz'
[M::worker_pipeline::3.579*1.98] mapped 645800 sequences
[M::main] Version: 2.17-r941
[M::main] CMD: minimap2 -c -a ./files/step_2_read_mapping/chr19_minimap2_index ./files/step_1_ngs_quality_control/QC_by_fastp/./files/step_1_ngs_quality_control/QC_by_fastp/QC_SRR075914_1.fastq.gz ./files/step_1_ngs_quality_control/QC_by_fastp/QC_SRR075914_2.fastq.gz
[M::main] Real time: 3.620 sec; CPU: 7.125 sec; Peak RSS: 0.476 GB


In [4]:
# check the output file
!head ./files/step_2_read_mapping/SRR075914_chr19_minimap2.sam

@SQ	SN:chr19	LN:58617616
@PG	ID:minimap2	PN:minimap2	VN:2.17-r941	CL:minimap2 -c -a ./files/step_2_read_mapping/chr19_minimap2_index ./files/step_1_ngs_quality_control/QC_by_fastp/./files/step_1_ngs_quality_control/QC_by_fastp/QC_SRR075914_1.fastq.gz ./files/step_1_ngs_quality_control/QC_by_fastp/QC_SRR075914_2.fastq.gz
SRR075914.4	4	*	0	0	*	*	0	0	TGGGTGGGAACACAGCCAAACCATATTAGGGGCACAAGAGAACTTTCAGGTATGATAGANATGTTNNN	A>>:;CA=:C5DD5DEE??EEBAEED5BDA)*567=DDCA5A<<?AAA5:==C######!#####!!!	rl:i:0
SRR075914.5	4	*	0	0	*	*	0	0	TATAACAATCATGCAAGAGAGCTTCTATTGTTATCATCCCCACTTCACAGAGAATAAAACTGAGACAN	DGGFDGGGGDGGGAGD?A?:A<AA?FFFDDCD?CA>?-A(AA:A?EB-=BCBBD-CB##########!	rl:i:0
SRR075914.13	4	*	0	0	*	*	0	0	TCCCTGGATAAAAATACTCCTTCCTGTAGTCTCTCAATTCAGAGCTCATAGTCAGTCTTTTCTCTCCT	EEAD:EE=EEFDFE8CCEEDGGB=BFDFFDDEEABED?5:DD=:CABBEEEDEE?FFD:FC@6,CCCC	rl:i:0
SRR075914.18	4	*	0	0	*	*	0	0	GTCAGTCTCAGAGTAGATATGGTAAGAACTTTGGACCCATGGACAGAAACGTTCCATTTGAACTATCC	GGGDFGGGGFEDEBAEE?=EFF:F?FFFFAGG=GDC-AAA@C-CBA<?&?C>>.@#####

In [5]:
!wc -l ./files/step_2_read_mapping/SRR075914_chr19_minimap2.sam

697290 ./files/step_2_read_mapping/SRR075914_chr19_minimap2.sam


---

### 4. Discussion

In this demonstration, I ran only bwa-mem and minimap2. However, we can see that there is difference in number of mapped reads. There are studies comparing the efficiency of different read mappers such as this one https://doi.org/10.3390/plants9040439
